<a href="https://colab.research.google.com/github/garifollinad/CO2Predict/blob/main/Train_script_thesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install streamlit
# import streamlit as st

from sklearn.linear_model import Lasso, LinearRegression
# from sklearn import svm
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn import tree
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import TimeSeriesSplit
# from sklearn.preprocessing import StandardScaler

from pandas.tseries.offsets import MonthEnd, MonthBegin
from datetime import datetime
import holidays
import time
import copy
import itertools

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
def read_file(file):
    employees_data = pd.read_excel(file, sheet_name='Employees')
    trips_data = pd.read_excel(file, sheet_name='Trips')
    return employees_data, trips_data

In [3]:
employees_data, trips_data = read_file('/content/FynchData_train.xlsx')

In [4]:
def transform_date_columns(trips_data):
    dw_mapping={
        0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday',
        4: 'Friday', 5: 'Saturday', 6: 'Sunday'
    }

    trips_data['TimeDuration'] = round((trips_data.StopTime - trips_data.StartTime) / np.timedelta64(1, 'h'), 2)
    trips_data['Date'] = trips_data.StartTime.dt.date
    trips_data['StartTime'] = trips_data.StartTime.dt.time
    trips_data['EndTime'] = trips_data.StopTime.dt.time
    trips_data.drop(columns='StopTime', inplace=True)
    trips_data['Date'] = pd.to_datetime(trips_data['Date'])
    trips_data['Day_of_week'] = trips_data.Date.dt.weekday.map(dw_mapping)
    return trips_data

In [5]:
trips_data = transform_date_columns(trips_data)

In [6]:
def get_stat_by_group(group_column, stat_columns, df):
    group_names = df[group_column].unique()
    info = pd.DataFrame(columns=['group', 'Q1', 'mean', 'median', 'Q3', 'IQR', 'count', 'min_val', 'max_val'])
    for g in group_names:
        data = df.loc[df[group_column] == g]
        q1 = data[stat_columns].quantile(0.25)
        mean = data[stat_columns].mean()
        median = data[stat_columns].quantile(0.5)
        q3 = data[stat_columns].quantile(0.75)
        iqr = q3 - q1
        min_val = q1 - 1.5 * iqr
        max_val = q3 + 1.5 * iqr
        info.loc[len(info)] = [g, q1, mean, median, q3, iqr, len(data), min_val, max_val]
    return info

def delete_outliers(commute_trips):
    commute_trips = commute_trips.loc[commute_trips['Co2_emissions'] > 0]
    commute_trips = commute_trips.loc[commute_trips['TimeDuration'] < 4]
    commute_trips = commute_trips[~commute_trips.Day_of_week.isin(['Saturday', 'Sunday'])]

    commute_trips.loc[(commute_trips['FuelType'].notnull()) &
               (commute_trips['Vehicle'].isin(['Bike', 'Foot', 'SharedBike'])), 'FuelType'] = np.NaN
    commute_trips['VehicleWithFuel'] = np.where(
        (~commute_trips['FuelType'].isnull()) & (commute_trips['Vehicle'].str.contains('Car')),
        commute_trips['Vehicle'][(~commute_trips['FuelType'].isnull()) & (commute_trips['Vehicle'].str.contains('Car'))] + ' ' + commute_trips['FuelType'],
        commute_trips['Vehicle'])
    commute_trips = commute_trips.loc[~commute_trips['Vehicle'].isin(['Plane', 'Boat', 'SharedElectricCar', 'Motorbike'])]
    commute_trips['Vehicle'] = np.where((commute_trips['FuelType'] == 'electrisch') & (commute_trips['Vehicle'].str.contains('Car')), 'ElectricCar', commute_trips['Vehicle'])

    commute_trips['Emission by km'] = round(commute_trips['Co2_emissions'] / commute_trips['Distance'], 2)
    vehicle_emissions_km_info = get_stat_by_group('Vehicle', 'Emission by km', commute_trips)
    outliers_by_emission_km = pd.DataFrame(columns=commute_trips.columns)
    for i in range(len(vehicle_emissions_km_info)):
        data = commute_trips.loc[(commute_trips['Vehicle'] == vehicle_emissions_km_info.iloc[i][0]) &
                                 ((commute_trips['Emission by km'] < vehicle_emissions_km_info.iloc[i][7]) | (commute_trips['Emission by km'] > vehicle_emissions_km_info.iloc[i][8]))]
        outliers_by_emission_km = outliers_by_emission_km.append(data)
    commute_trips = commute_trips.drop(outliers_by_emission_km.index)

    commute_trips['Avg Velocity'] = commute_trips['Distance'] // commute_trips['TimeDuration']
    max_avg_velocity= {'Car': 120, 'Bike': 25, 'Foot': 15, 'PublicTransport': 70,
                       'Carpool': 120, 'ElectricBike': 25, 'Train': 150,
                       'ElectricCar': 120, 'SharedBike': 25, 'Moped': 50}
    vehicle_velocity_info = get_stat_by_group('Vehicle', 'Avg Velocity', commute_trips)
    outliers_by_velocity = pd.DataFrame(columns=commute_trips.columns)
    for i in range(len(vehicle_velocity_info)):
        human_max_velocity = max_avg_velocity[vehicle_velocity_info.iloc[i][0]]
        min_val, max_val = vehicle_velocity_info.iloc[i][7], max(human_max_velocity,  vehicle_velocity_info.iloc[i][8])
        data = commute_trips.loc[(commute_trips['Vehicle'] == vehicle_velocity_info.iloc[i][0]) &
                                 ((commute_trips['Avg Velocity'] < min_val) | (commute_trips['Avg Velocity'] > max_val))]
        outliers_by_velocity = outliers_by_velocity.append(data)
    commute_trips = commute_trips.drop(outliers_by_velocity.index)
    return commute_trips

In [7]:
trips_data = trips_data[['EmployeeId', 'TripId', 'TripType', 'Vehicle', 'Date', 'Day_of_week', 'StartTime', 'EndTime', 'TimeDuration', 'Distance', 'Co2_emissions']]
trips_data = pd.merge(trips_data, employees_data[['EmployeeId', 'CompanyId', 'FuelType']], on='EmployeeId', how='left')
commute_trips = trips_data.loc[trips_data['TripType'] == 'commute']
updated_commute_trips = delete_outliers(commute_trips)
print(updated_commute_trips.columns)

Index(['EmployeeId', 'TripId', 'TripType', 'Vehicle', 'Date', 'Day_of_week',
       'StartTime', 'EndTime', 'TimeDuration', 'Distance', 'Co2_emissions',
       'CompanyId', 'FuelType', 'VehicleWithFuel', 'Emission by km',
       'Avg Velocity'],
      dtype='object')


In [8]:
def lag_feature(df_grouped, shift_months, cols):
  for c in cols:
    tmp = df_grouped[['Date', 'CompanyId', 'VehicleWithFuel', c]]
    for i in range(1, shift_months+1):
      shifted = tmp.copy()
      shifted.columns = ['Date', 'CompanyId', 'VehicleWithFuel', c + "_lag_"+str(i)]
      shifted.Date = shifted.Date + pd.DateOffset(months=i)
      df_grouped = pd.merge(df_grouped, shifted, on=['Date', 'CompanyId', 'VehicleWithFuel'], how='left')
  return df_grouped

def count_workdays(df):
    h = holidays.Netherlands()
    b = pd.bdate_range(df['Date'], df['Date'] + MonthEnd(0))
    return sum(y not in h for y in b)

def train_valid_split(df):
  months = df.index.unique()
  n_months = len(months)
  # print(months)
  if len(months) < 2:
    raise ValueError("At least should be two months data")

  for m in range(1, n_months-1):
    test_idx = months[m]
    train_idx = months[:m]

    yield train_idx, test_idx

def create_models(type, params, train_df_len=0):
  new_params = copy.deepcopy(params)
  models = [None] * len(new_params)
  for i in range(len(new_params)):
    if type == 'HistGradientBoostingRegressor':
      new_val = max(1, round(new_params[i]['min_samples_leaf'] * train_df_len))
      new_params[i]['min_samples_leaf'] = new_val

      reg = HistGradientBoostingRegressor()
    else: # type == 'RandomForestRegressor'
      reg = RandomForestRegressor()

    reg.set_params(**new_params[i])
    models[i] = reg
  return models
    # print(valid_vals)

# def scale_needed_columns(scaler, X, cols, only_transform=True):
#   other_df = X[~X.columns.isin(cols)]
#   transform_df = X[X.columns.isin(cols)]

#   if only_transform:
#     transformed_df = scaler.transform(transform_df)
#   else:
#     transformed_df = scaler.fit_transform(transform_df)

#   output_df = pd.concat([transformed_df, other_df], axis=1)
#   return output_df


In [82]:
def train_model(updated_commute_trips):
  model_data = updated_commute_trips.copy()
  model_data.drop(columns=['EmployeeId', 'TripId', 'TripType', 'Vehicle', 'FuelType', 'Day_of_week', 'EndTime', 'Emission by km', 'Avg Velocity'], inplace=True)
  model_data['StartTime'] = model_data['StartTime'].apply(lambda x: round(x.hour + x.minute/60, 2))

  model_data['Date'] = model_data['Date'] + MonthEnd(0) - MonthBegin(1)

  model_data = model_data.loc[model_data['Date'] > '2022-04-01', :]

  model_data_monthly = model_data.copy()
  model_data_monthly.sort_values(['VehicleWithFuel', 'CompanyId', 'Date'], inplace=True)

  model_data_monthly = model_data_monthly.groupby(['Date', 'CompanyId', 'VehicleWithFuel'], as_index=False)

  model_data_monthly = model_data_monthly.agg({'TimeDuration': ['mean'], 'Distance': ['mean'],
                          'Co2_emissions': ['mean', 'sum', 'count']})
  model_data_monthly.columns = ['Date', 'CompanyId', 'VehicleWithFuel',
                                'TimeDuration_mean', 'Distance_mean', 'Co2_mean', 'Co2_sum', 'Count']

  model_data_monthly_with_lag = lag_feature(model_data_monthly, 3,
      ['TimeDuration_mean', 'Distance_mean', 'Co2_mean', 'Count'])

  model_data_monthly_with_lag.fillna(0, inplace=True)
  model_data_monthly_with_lag['qmean_cnt'] = model_data_monthly_with_lag[['Count_lag_1',
                                  'Count_lag_2',
                                  'Count_lag_3']].mean(skipna=True, axis=1)
  # Add quater std count
  model_data_monthly_with_lag['qstd_cnt'] = model_data_monthly_with_lag[['Count_lag_1',
                                      'Count_lag_2',
                                      'Count_lag_3']].std(skipna=True, axis=1)
  # Add quater min count
  model_data_monthly_with_lag['qmin_cnt'] = model_data_monthly_with_lag[['Count_lag_1',
                                      'Count_lag_2',
                                      'Count_lag_3']].min(skipna=True, axis=1)
  # Add quater max count
  model_data_monthly_with_lag['qmax_cnt'] = model_data_monthly_with_lag[['Count_lag_1',
                                      'Count_lag_2',
                                      'Count_lag_3']].max(skipna=True, axis=1)

  ohe = OneHotEncoder()
  # le = LabelEncoder()
  ohe_encode_columns = ['VehicleWithFuel']
  # le_encode_columns = ['CompanyId']
  ohe_encoded_array = ohe.fit_transform(model_data_monthly_with_lag[ohe_encode_columns]).toarray()
  ohe_encoded_labels = ohe.get_feature_names_out()
  ohe_encoded_df = pd.DataFrame(ohe_encoded_array, columns=ohe_encoded_labels)
  model_data_monthly_with_lag.drop(columns=ohe_encode_columns, inplace=True)
  model_data_monthly_with_lag = pd.concat([model_data_monthly_with_lag, ohe_encoded_df], axis=1)
  model_data_monthly_with_lag.dropna(inplace=True)
  # model_data_monthly_with_lag['CompanyId'] = le.fit_transform(model_data_monthly_with_lag[le_encode_columns])

  nl_holidays = holidays.Netherlands()
  months = model_data_monthly_with_lag.index.unique()
  model_data_monthly_with_lag['Nb_work_days'] =  model_data_monthly_with_lag.apply(count_workdays, axis=1)

  model_data_monthly_with_lag['Year'] = model_data_monthly_with_lag['Date'].dt.year
  model_data_monthly_with_lag['Month'] = model_data_monthly_with_lag['Date'].dt.month
  model_data_monthly_with_lag['Week'] = model_data_monthly_with_lag['Date'].dt.week


  # For count predict model
  count_columns = ['Year', 'Month', 'Week',
      'Count_lag_1', 'Count_lag_2', 'Count_lag_3',
      'qmean_cnt', 'qstd_cnt', 'qmin_cnt', 'qmax_cnt', 'Nb_work_days']
  # scale_count_columns = count_columns.copy()
  ohe_encoded_labels_list = ohe_encoded_labels.tolist()
  count_columns.extend(ohe_encoded_labels_list)

  # For co2_mean predict model
  co2_columns = ['Year', 'Month', 'Week',
      'TimeDuration_mean_lag_1', 'TimeDuration_mean_lag_2','TimeDuration_mean_lag_3',
      'Distance_mean_lag_1', 'Distance_mean_lag_2', 'Distance_mean_lag_3',
      'Co2_mean_lag_1', 'Co2_mean_lag_2','Co2_mean_lag_3']
  # scale_co2_columns = co2_columns.copy()
  co2_columns.extend(ohe_encoded_labels_list)

  vehicle_columns_f = 'vehicle_columns.sav'
  pickle.dump(ohe_encoded_labels_list, open(vehicle_columns_f, 'wb'))

  model_data_monthly_with_lag.set_index('Date', inplace=True)

  r1 = LinearRegression()
  r1_models = [r1]
  # results1 = {}

  # r2 = HistGradientBoostingRegressor()
  param2 = {
      'loss': ['squared_error'],
      'learning_rate': [0.5, 1],
      'max_iter': [1, 4, 16],
      'max_depth': [2, 4],
      'min_samples_leaf': [0.2, 0.6],
      'l2_regularization': [0, 0.1, 0.01]
  }
  keys, values = zip(*param2.items())
  r2_model_params = [dict(zip(keys, v)) for v in itertools.product(*values)]
  # r2_models = create_models('HistGradientBoostingRegressor', r2_model_params)
  # results2 = {}

  # r3 = RandomForestRegressor()
  param3 = {
      'n_estimators': [100, 200, 300, 400],
      'bootstrap': [True, False],
      'max_depth': [5, 10, 20],
      'min_samples_leaf': [4, 8, 16],
      'min_samples_split': [5, 10]
  }
  keys, values = zip(*param3.items())
  r3_model_params = [dict(zip(keys, v)) for v in itertools.product(*values)]
  r3_models = create_models('RandomForestRegressor', r3_model_params)

  model_res = pd.DataFrame(columns=['Model', 'Model_index', 'Target', 'CompanyId', 'Iter', 'RMSE', 'MAE', 'r2', 'Adj_r2', 'Time'])

  for c in model_data_monthly_with_lag['CompanyId'].unique():
    data = model_data_monthly_with_lag.loc[model_data_monthly_with_lag['CompanyId'] == c, :]

    y_train_valid = data['Co2_sum']
    y_count = data['Count']
    X_count = data.loc[:, count_columns]
    y_co2 = data['Co2_mean']
    X_co2 = data.loc[:, co2_columns]

    for iter, (train_index, valid_index) in enumerate(train_valid_split(data)):
      print(iter)
      X_train_count, X_valid_count = X_count.loc[train_index, :], X_count.loc[valid_index,:]
      # X_train_count_scaled, X_valid_count_scaled = X_count_scaled.iloc[train_index, :], X_count_scaled.iloc[test_index,:]
      y_train_count, y_valid_count = y_count.loc[train_index], y_count.loc[valid_index]
      X_train_co2, X_valid_co2 = X_co2.loc[train_index, :], X_co2.loc[valid_index,:]
      # X_train_co2_scaled, X_valid_co2_scaled = X_co2_scaled.iloc[train_index, :], X_co2_scaled.iloc[test_index,:]
      y_train_co2, y_valid_co2 = y_co2.loc[train_index], y_co2.loc[valid_index]
      y_train, y_valid = y_train_valid.loc[train_index], y_train_valid.loc[valid_index]

      r2_models = create_models('HistGradientBoostingRegressor', r2_model_params, X_train_count.shape[0])

      for i, r1 in enumerate(r1_models):
        print('LinearRegression', i)
        t0 = time.time()
        r1.fit(X_train_count, y_train_count)
        t = time.time()-t0
        y_predict = r1.predict(X_valid_count)
        rmse = mean_squared_error(y_valid_count, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_count, y_predict)
        r2_s = r2_score(y_valid_count, y_predict)
        n, p = X_valid_count.shape[0], X_valid_count.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['LinearRegression', i, 'Count', c, iter, rmse, mae, r2_s, adj_r2, t]

        t0 = time.time()
        r1.fit(X_train_co2, y_train_co2)
        t = time.time()-t0
        y_predict = r1.predict(X_valid_co2)
        rmse = mean_squared_error(y_valid_co2, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_co2, y_predict)
        r2_s = r2_score(y_valid_co2, y_predict)
        n, p = X_valid_co2.shape[0], X_valid_co2.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['LinearRegression', i, 'Co2_mean', c, iter, rmse, mae, r2_s, adj_r2, t]

      for i, r2 in enumerate(r2_models):
        print('HistGradientBoostingRegressor', i)
        t0 = time.time()
        r2.fit(X_train_count, y_train_count)
        t = time.time()-t0
        y_predict = r2.predict(X_valid_count)
        rmse = mean_squared_error(y_valid_count, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_count, y_predict)
        r2_s = r2_score(y_valid_count, y_predict)
        n, p = X_valid_count.shape[0], X_valid_count.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['HistGradientBoostingRegressor', i, 'Count', c, iter, rmse, mae, r2_s, adj_r2, t]

        t0 = time.time()
        r2.fit(X_train_co2, y_train_co2)
        t = time.time()-t0
        y_predict = r2.predict(X_valid_co2)
        rmse = mean_squared_error(y_valid_co2, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_co2, y_predict)
        r2_s = r2_score(y_valid_co2, y_predict)
        n, p = X_valid_co2.shape[0], X_valid_co2.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['HistGradientBoostingRegressor', i, 'Co2_mean', c, iter, rmse, mae, r2_s, adj_r2, t]

      for i, r3 in enumerate(r3_models):
        print('RandomForestRegressor', i)
        t0 = time.time()
        r3.fit(X_train_count, y_train_count)
        t = time.time()-t0
        y_predict = r3.predict(X_valid_count)
        rmse = mean_squared_error(y_valid_count, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_count, y_predict)
        r2_s = r2_score(y_valid_count, y_predict)
        n, p = X_valid_count.shape[0], X_valid_count.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['RandomForestRegressor', i, 'Count', c, iter, rmse, mae, r2_s, adj_r2, t]

        t0 = time.time()
        r3.fit(X_train_co2, y_train_co2)
        t = time.time()-t0
        y_predict = r3.predict(X_valid_co2)
        rmse = mean_squared_error(y_valid_co2, y_predict, squared=False)
        mae = mean_absolute_error(y_valid_co2, y_predict)
        r2_s = r2_score(y_valid_co2, y_predict)
        n, p = X_valid_co2.shape[0], X_valid_co2.shape[1]
        adj_r2 = 1 - (1 - r2_s) * ((n - 1)/(n - p - 1))
        model_res.loc[len(model_res)] = ['RandomForestRegressor', i, 'Co2_mean', c, iter, rmse, mae, r2_s, adj_r2, t]

  results_copy = model_res.copy()
  scaled_results = pd.DataFrame(columns = ['Model', 'Model_index', 'Target', 'CompanyId', 'Iter', 'RMSE', 'MAE', 'r2', 'Adj_r2', 'Time'])
  cols = scaled_results.columns.tolist()

  scaler = Pipeline(steps=[
          ('standard', RobustScaler())])

  for t in results_copy['Target'].unique():
    for c in results_copy['CompanyId'].unique():
      scale_data = results_copy.loc[(results_copy['CompanyId'] == c) & (results_copy['Target'] == t), :]
      ct = ColumnTransformer(transformers=[('std', scaler , ['RMSE', 'MAE'])],
                          remainder='passthrough')
      res = pd.DataFrame(ct.fit_transform(scale_data),
                        columns = ['RMSE', 'MAE', 'Model', 'Model_index', 'Target', 'CompanyId', 'Iter', 'r2', 'Adj_r2', 'Time'])
      res = res[cols[:2] + cols[2:]]
      scaled_results = pd.concat([scaled_results, res], axis=0)

  scaled_results.rename(columns={'RMSE': 'RMSE_sc', 'MAE': 'MAE_sc'}, inplace=True)

  scaled_results.drop(['CompanyId', 'Iter'], axis=1, inplace=True)
  final_res = scaled_results.groupby(['Model', 'Model_index', 'Target'], as_index=False).mean()

  final_res['Error'] = 0.7 * final_res['RMSE_sc'] + 0.3 * final_res['MAE_sc']
  for t in final_res['Target'].unique():
    top_model = final_res.loc[final_res['Target'] == t, :].sort_values('Error').head(1)
    model_type, model_idx = top_model['Model'].values[0], top_model['Model_index'].values[0]

    if model_type == 'HistGradientBoostingRegressor':
      model = HistGradientBoostingRegressor()
      model.set_params(**r2_model_params[model_idx])
    elif model_type == 'RandomForestRegressor':
      model = RandomForestRegressor()
      model.set_params(**r3_model_params[model_idx])
    else:
      model = LinearRegression()

    model_f = f'model_{t}.sav'
    pickle.dump(model, open(model_f, 'wb'))

In [ ]:
train_model(updated_commute_trips)